In [0]:
%pip install dbldatagen

In [0]:
from pyspark.sql.functions import expr

row_count = 500
channel_types = ["Online", "Offline"]
regions = ["Asia", "Europe", "Americas", "Africa", "Australia"]

from dbldatagen import DataGenerator as dgI

store_df = (
    dgI(spark, rows=row_count)
    .withColumn("channel_type", values=channel_types, random=True)
    .withColumn("region", values=regions, random=True)
    .withColumn(
        "store_sk",
        "int",
        minValue=1,
        maxValue=row_count,
        step=1,
        random=False,
        nullable=False,
    )
    .withColumn(
        "store_id", "string", expr="concat('S', lpad(cast(store_sk as string), 3, '0'))"
    )
    .build()
)


store_df = store_df.withColumn(
    "name",
    expr(
        """
        concat(
         CASE region
          WHEN 'Asia' THEN 'AS'
          WHEN 'Europe' THEN 'EU'
          WHEN 'Americas' THEN 'AM'
          WHEN 'Africa' THEN 'AF'
          WHEN 'Australia' THEN 'AU'
          ELSE 'XX'
        END,
        ' ',
 
            substring('ABCDEFGHIJKLMNOPQRSTUVWXYZ', floor(rand() * 26) + 1, 1),
            substring('abcdefghijklmnopqrstuvwxyz', floor(rand() * 26) + 1, 3)
        )
        """
    ),
)

In [0]:
store_df.write.mode("overwrite").saveAsTable("dev.bronze.dim_store")